# I. Выпуклость, необходимое условие минимума, двойственная задача

## 1. Критерий выпуклости второго порядка

Optimal allocation with resource constraint:
$$f_{i}\left(x_{i}\right) = a_{i}e^{x_{i}}, a_{i} > 0, i \in \{ 1, ..., n\}$$

Также ограничение бюджета:
$$\sum^{n}_{i=1}x_{i} = b, b \in R$$

Задача минимизации:
$$\sum^{n}_{i=1}a_{i}e^{x_{i}} \rightarrow min, a_{i} > 0, i \in \{ 1, ..., n\}$$

Итоговая задача:
$$\sum^{n}_{i=1}a_{i}e^{x_{i}} \rightarrow min, a_{i} > 0, i \in \{ 1, ..., n\}, \sum^{n}_{i=1}x_{i} = b, b \in R$$

Для изучения выпуклости рассмотрим гессиан оптимизируемой функции:
$$H(x)_{i,j} = 0, i \neq j$$
$$H(x)_{i,j} = a_{j}e^{x_{j}}, i = j$$
$$\text{При } a_{i} > 0 \text{ очевидно, что } H(x)_{i,i} = a_{i}e^{x_{i}} > 0, \text{ так как } \forall x \in R: e^{x_{i}} > 0$$
Матрица такого вида является положительно определенной.

Так же рассмотрим знакоопределенность гессиана ограничений:
$$H(x) = \textbf{0}$$ 

Таким образом, задача является выпуклой.

## 2. Условие оптимальности

Рассмотрим Лангранжиан системы:
$$L(x, \nu) = \sum^{n}_{i=1}a_{i}e^{x_{i}} + \nu (\textbf{1}^Tx-b)$$

Для потимальности найдем точки нуля производной лагранжиана, такие точки отметим звездой:
$$\nabla L (x^{\star}, \nu^{\star}) = \left( a_{1}e^{x_{1}^{\star}}, ... , a_{n}e^{x_{n}^{\star}} \right)^T + {\boldsymbol\nu^{\star}}^T  = 0$$

Выражая покомпонентно элементы вектора x*:
$$a_{i}e^{x_{i}^{\star}} = -\nu^{\star}, i \in \{ 1, ..., n\}$$
$$x_{i}^{\star} = ln\left( - \frac{\nu^{\star}}{a_{i}}\right), i \in \{ 1, ..., n\}$$

Таким образом условия оптимума ККТ будут иметь вид следующей системы:

\begin{cases}
  x_{i}^{\star} = ln\left( - \frac{\nu^{\star}}{a_{i}}\right), i \in \{ 1, ..., n\}\\
  \textbf{1}^Tx^{\star} = b
\end{cases}

## 3. Двойственная задача

Двойственной к оригинальной задачей будет инфинум лагранжиана по вектору переменных x.

$$g(\nu) = \underset{x}{inf} L(x, \nu)$$
$$g(\nu) = -n \nu + \nu \left( \sum^{n}_{i=1}ln\left( -\frac{\nu}{a_{i}}\right) -b \right) = \nu \left( \sum^{n}_{i=1}ln\left( -\frac{\nu}{a_{i}}\right) -n -b \right)$$

Тогда двойственная задача есть:
$$\nu \left( \sum^{n}_{i=1}ln\left( -\frac{\nu}{a_{i}}\right) -n -b \right) \rightarrow max$$

# II. Решение тестовых примеров

In [1]:
import numpy as np
import cvxpy as cp
import time
import polars as pl
import matplotlib.pyplot as plt
import os

## 1. Генерация примеров

In [2]:
n_dims = range(10, 110, 10)
n_examples = 100
n_seeds = 100

In [3]:
raw_experiment_sheet = pl.DataFrame({"dim": n_dims}).select(
    pl.all().repeat_by(n_examples).flatten()
)

In [ ]:
raw_experiment_sheet = raw_experiment_sheet.with_columns(
    pl.col("dim")
    .map_elements(
        lambda dim_: np.random.uniform(low=0.1, high=100, size=dim_),
        return_dtype=pl.List(float),
        returns_scalar=False,
    )
    .alias("a"),
    pl.col("dim")
    .map_elements(
        lambda dim_: np.random.uniform(low=-100, high=100, size=1),
        return_dtype=float,
        returns_scalar=True,
    )
    .alias("b"),
)

In [5]:
raw_experiment_sheet

dim,a,b
i64,list[f64],f64
10,"[98.623768, 17.622572, … 76.756343]",-1.377702
10,"[87.46204, 71.092403, … 40.224658]",-76.654283
10,"[78.127373, 6.493261, … 45.204324]",-0.595597
10,"[94.079596, 43.463478, … 65.397703]",92.569535
10,"[59.991563, 99.909557, … 12.030155]",43.24865
…,…,…
100,"[4.327401, 10.366394, … 27.713563]",-76.934482
100,"[55.736813, 79.941932, … 31.637637]",68.342216
100,"[55.764994, 88.193946, … 12.331152]",-59.567157


## 2. Решение солвером CVXPY

In [ ]:
def solve_oawrc(a, b):

    a = np.asarray(a)

    x = cp.Variable(a.shape[0])

    objective = cp.Minimize(cp.sum(a @ cp.exp(x)))

    constraints = [cp.sum(x) - b == 0]

    prob = cp.Problem(objective, constraints)

    try:
        start_t = time.time()
        result = prob.solve()
        sovle_time = time.time() - start_t
    except:
        res = {"cvxpy_x": None, "cvxpy_y": None, "cvxpy_t": None}
    else:
        res = {"cvxpy_x": list(x.value), "cvxpy_y": result, "cvxpy_t": sovle_time}

    return res

In [ ]:
raw_experiment_sheet = raw_experiment_sheet.with_columns(
    (
        pl.struct("a", "b")
        .map_elements(
            lambda x: solve_oawrc(**x),
            return_dtype=pl.Struct(
                {"cvxpy_x": pl.List(float), "cvxpy_y": float, "cvxpy_t": float}
            ),
        )
        .alias("cvxpy_out")
    )
).unnest("cvxpy_out")

/home/igkh/.venv/lib/python3.12/site-packages/cvxpy/problems/problem.py:1481: UserWarning: Solution may be inaccurate. Try another solver, adjusting the solver settings, or solve with verbose=True for more information.
  warnings.warn(


In [8]:
raw_experiment_sheet

dim,a,b,cvxpy_x,cvxpy_y
i64,list[f64],f64,list[f64],f64
10,"[98.623768, 17.622572, … 76.756343]",-1.377702,"[-1.047906, 0.674315, … -0.797123]",345.848778
10,"[87.46204, 71.092403, … 40.224658]",-76.654283,"[-8.693696, -8.486249, … -7.916634]",0.146666
10,"[78.127373, 6.493261, … 45.204324]",-0.595597,"[-1.053569, 1.434052, … -0.506469]",272.417762
10,"[94.079596, 43.463478, … 65.397703]",92.569535,"[8.027426, 8.799692, … 8.391119]",2.8823e6
10,"[59.991563, 99.909557, … 12.030155]",43.24865,"[4.326456, 3.816393, … 5.933241]",45399.739668
…,…,…,…,…
100,"[4.327401, 10.366394, … 27.713563]",-76.934482,"[1.382804, 0.509202, … -0.474151]",1724.92786
100,"[55.736813, 79.941932, … 31.637637]",68.342216,"[0.305232, -0.055428, … 0.871495]",7563.079362
100,"[55.764994, 88.193946, … 12.331152]",-59.567157,"[-1.029215, -1.487623, … 0.479811]",1992.425675


## 3. Решение прямой задачи методом Ньютона 

In [9]:
def random_sum_bound(dim, b):
    rnd = np.random.uniform(size=dim)
    return list(rnd / rnd.sum() * b)

In [10]:
raw_experiment_sheet = raw_experiment_sheet.select(
    pl.all().repeat_by(n_seeds).flatten()
).with_columns(
    pl.struct("dim", "b")
    .map_elements(lambda x: random_sum_bound(**x), return_dtype=pl.List(float))
    .alias("x_init")
)

In [11]:
raw_experiment_sheet

dim,a,b,cvxpy_x,cvxpy_y,x_init
i64,list[f64],f64,list[f64],f64,list[f64]
10,"[98.623768, 17.622572, … 76.756343]",-1.377702,"[-1.047906, 0.674315, … -0.797123]",345.848778,"[-0.178878, -0.074887, … -0.141447]"
10,"[98.623768, 17.622572, … 76.756343]",-1.377702,"[-1.047906, 0.674315, … -0.797123]",345.848778,"[-0.041772, -0.037545, … -0.139961]"
10,"[98.623768, 17.622572, … 76.756343]",-1.377702,"[-1.047906, 0.674315, … -0.797123]",345.848778,"[-0.158572, -0.213413, … -0.135807]"
10,"[98.623768, 17.622572, … 76.756343]",-1.377702,"[-1.047906, 0.674315, … -0.797123]",345.848778,"[-0.13982, -0.259188, … -0.213816]"
10,"[98.623768, 17.622572, … 76.756343]",-1.377702,"[-1.047906, 0.674315, … -0.797123]",345.848778,"[-0.10706, -0.1904, … -0.139037]"
…,…,…,…,…,…
100,"[34.487485, 21.377496, … 33.723074]",81.846787,"[0.793328, 1.271548, … 0.815723]",7624.461176,"[1.405423, 0.823007, … 0.667615]"
100,"[34.487485, 21.377496, … 33.723074]",81.846787,"[0.793328, 1.271548, … 0.815723]",7624.461176,"[0.361587, 0.005409, … 0.332045]"
100,"[34.487485, 21.377496, … 33.723074]",81.846787,"[0.793328, 1.271548, … 0.815723]",7624.461176,"[0.464278, 1.362884, … 0.673694]"


## 4. Решение двойственной задачи методом Ньютона

# III. Анализ зависимостей

# IV. Выводы